### Reading Results.json file

In [0]:
dbutils.notebook.run("../Mounting_storage", 120)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType, TimestampType

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                            StructField("raceId", IntegerType(), True),
                            StructField("driverId", IntegerType(), True),
                            StructField("constructorId", IntegerType(), True),
                            StructField("number", IntegerType(), True),
                            StructField("grid", IntegerType(), True),
                            StructField("position", IntegerType(), True),
                            StructField("positionText", StringType(), True),
                            StructField("positionOrder", IntegerType(), True),
                            StructField("points", DoubleType(), True),
                            StructField("laps", IntegerType(), True),
                            StructField("time", StringType(), True),
                            StructField("milliseconds", IntegerType(), True),
                            StructField("fastestLap", IntegerType(), True),
                            StructField("rank", IntegerType(), True),
                            StructField("fastestLapTime", StringType(), True),
                            StructField("fastestLapSpeed", DoubleType(), True),
                            StructField("statusId", StringType(), True)])


In [0]:
df_result = spark.read.json("/mnt/blobstorage/results.json", schema=results_schema)
display(df_result)

### Renaming Columns and add

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
df_results_with_column = df_result.withColumn("ingestion_date", current_timestamp())
display(df_results_with_column)

In [0]:
df_results_with_column = df_results_with_column.withColumnRenamed("resultId","result_id")\
                                  .withColumnRenamed("raceId","race_id")\
                                  .withColumnRenamed("driverId","driver_id")\
                                  .withColumnRenamed("constructorId","constructor_id")\
                                  .withColumnRenamed("positionText","position_text")\
                                  .withColumnRenamed("positionOrder","position_order")\
                                      .withColumnRenamed("fastestLap","fastest_lap")\
                                       .withColumnRenamed("fastestLapTime","fastest_lap_time")\
                                        .withColumnRenamed("fastestLapSpeed","fastest_lap_speed")
display(df_results_with_column)

### Drop unwanted columns

In [0]:
from pyspark.sql.functions import col

In [0]:
df_resul_final = df_results_with_column.drop(col("statusId"))

### Write the output to the parquet file

In [0]:
df_resul_final.write.mode("overwrite").partitionBy("race_id").parquet("/mnt/blobstorage/results")

In [0]:
%fs
ls mnt/blobstorage/results